In [23]:
import numpy as np
import pandas as pd
from numpy.fft import fft
from scipy import stats
from scipy.integrate import quad
from scipy.optimize import brute, fmin
import datetime

# **M1**

In [2]:
S0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2

## **Black-Scholes model**

In [3]:
# BS Model
def bs_analytic_call(S0, K, T, r, sigma):
    "BS model"
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return S0 * stats.norm.cdf(d1, 0, 1) - np.exp(-r * T) * K * stats.norm.cdf(d2, 0, 1)

In [4]:
bs_analytic_call(S0, K, T, r, sigma).round(4)

10.4506

## **Fourier Transform as in Lewis**

### **Black-Scholes Characteristics Function**  

$$\varphi^{BS}(u,T)=e^{(s_0 + (r-\frac{\sigma^2}{2}) T ) \, iu-\frac{\sigma^2}{2} u^2 T} = e^{((\frac{s_0}{T} + r - \frac{\sigma^2}{2} ) \, iu - \frac{1}{2} \sigma^2 u^2) T}$$

### **Intergrate in Lewis**

$$C_0 = S_0 - \frac{\sqrt{S_0K}e^{-rT}}{\pi}\int^\infty_0 Re [e^{izk}\varphi(z-i/2)] \frac{dz}{z^2 + 1/4}$$

In [5]:
def BS_characteristic_func(v, x0, T, r, sigma):
    """
    Computes general Black-Scholes model characteristic function
    to be used in Fourier pricing methods like Lewis (2001) and Carr-Madan (1999)
    """
    cf_values = np.exp(
        ((x0 / T + r - 0.5 * sigma**2) * 1j * v - 0.5 * sigma**2 * v**2) * T
    )
    return cf_values

In [25]:
def BS_integral(u, S0, K, T, r, sigma):
    """
    Expenssion for the integral in Lewis (2001)
    """
    cf_values = BS_characteristic_func(u - 1j * 0.5, 0.0, T, r, sigma)
    int_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * (np.log(S0 / K))) * cf_values).real
    )
    return int_value


def BS_call_Lewis(S0, K, T, r, sigma):
    """
    European Call option price in BS under Lewis (2001)
    """
    int_value = quad(lambda u: BS_integral(u, S0, K, T, r, sigma), 0, 100)[0]
    call_value = max(0, S0 - np.exp(-r * T) * (np.sqrt(S0 * K)) / np.pi * int_value)
    return call_value

In [26]:
print(
    "Fourier Call Option price under Lewis (2001) is $",
    BS_call_Lewis(S0, K, T, r, sigma).round(4),
)

Fourier Call Option price under Lewis (2001) is $ 10.4506


### **Fast Fourier Transform (FFT) for Option Pricing**

In [140]:
    K = 95
    S0 = 100
    k = np.log(K / S0)
    
    x0 = np.log(S0 / S0)
    
    g = 1  # Factor to increase accuracy
    N = g * 4096
    
    eps = (g * 150) ** -1
    
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    vo = eta * (u - 1)
    print((k+b)/eps)

2048.0


In [27]:
def BS_call_fft(S0, K, T, r, sigma):
    """
    European Call Option price in BS under FFT
    """

    k = np.log(K / S0)
    x0 = np.log(S0 / S0)
    g = 1  # Factor to increase accuracy
    N = g * 4096
    eps = (g * 150) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    vo = eta * (u - 1)

    # Modification to ensure integrability
    # ITM
    if S0 >= 0.95 * K:
        alpha = 1.5
        v = vo - (alpha + 1) * 1j
        
        modcharFunc = np.exp(-r * T) * (
            BS_characteristic_func(v, x0, T, r, sigma)
            / (alpha**2 + alpha - vo**2 + 1j * (2 * alpha + 1) * vo)
        )
    # OTM
    else:
        alpha = 1.1
        v = (vo - 1j * alpha) - 1j
        modcharFunc1 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo - 1j * alpha))
            - np.exp(r * T) / (1j * (vo - 1j * alpha))
            - BS_characteristic_func(v, x0, T, r, sigma)
            / ((vo - 1j * alpha) ** 2 - 1j * (vo - 1j * alpha))
        )

        v = (vo + 1j * alpha) - 1j

        modcharFunc2 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo + 1j * alpha))
            - np.exp(r * T) / (1j * (vo + 1j * alpha))
            - BS_characteristic_func(v, x0, T, r, sigma)
            / ((vo + 1j * alpha) ** 2 - 1j * (vo + 1j * alpha))
        )

    delt = np.zeros(N)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    # ITM
    if S0 >= 0.95 * K:
        FFTFunc = np.exp(1j * b * vo) * modcharFunc * eta * SimpsonW
        payoff = (fft(FFTFunc)).real
        CallValueM = np.exp(-alpha * k) / np.pi * payoff
    # OTM
    else:
        FFTFunc = (
            np.exp(1j * b * vo) * (modcharFunc1 - modcharFunc2) * 0.5 * eta * SimpsonW
        )
        payoff = (fft(FFTFunc)).real
        CallValueM = payoff / (np.sinh(alpha * k) * np.pi)

    pos = int((k + b) / eps)
    CallValue = CallValueM[pos] * S0

    return CallValue

In [28]:
BS_call_fft(S0, K, T, r, sigma).round(4)

10.4506

In [115]:
begin = datetime.datetime.utcnow()
bs_analytic_call(S0, K, T, r, sigma)
end = datetime.datetime.utcnow()
x = end - begin
print(x)

0:00:00.001001


### **Heston Characteristic Function**

$$ \varphi^H(u, T) = e^{H_1(u,T)+H_2(u,T)v_0}$$

In [18]:
def H93_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    c1 = kappa_v * theta_v
    c2 = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
    )
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) / (
        kappa_v - rho * sigma_v * u * 1j - c2
    )
    H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
    )
    H2 = (
        (kappa_v - rho * sigma_v * u * 1j + c2)
        / sigma_v**2
        * ((1 - np.exp(c2 * T))
        / (1 - c3 * np.exp(c2 * T)))
    )

    char_func_value = np.exp(H1 + H2 * v0)

    return char_func_value

### **Integral Value in Lewis (2001)**

$$C_0 = S_0 - \frac{\sqrt{S_0 K} e^{-rT}}{\pi} \int_0^ \infty Re[e^{izk}\varphi(z-i/2)] \frac{dz}{z^2 + 1/4} $$

In [19]:
def H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    char_func_value = H93_char_func(
        u - 1j / 2, T, r, kappa_v, theta_v, sigma_v, rho, v0
    )
    int_func_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    )
    return int_func_value

In [13]:
def H93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    int_value = quad(
        lambda u: H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value

In [20]:
S0 = 100.0
K = 100.0
T = 1.0
r = 0.02

kappa_v = 1.5
theta_v = 0.02
sigma_v = 0.15
rho = 0.1
v0 = 0.01

H93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)

5.757807070918929

### **Heston model Calibration**

In [33]:
h5 = pd.HDFStore(
    "option_data_wqu.M1.h5", "r"
)
data = h5['data']
h5.close()
S0 = 3225.93

In [50]:
tol = 0.02
options = data[(np.abs(data["Strike"] - S0) / S0 ) < tol]
options["Date"] = pd.DatetimeIndex(options["Date"])
options["Maturity"] = pd.DatetimeIndex(options["Maturity"])

C:\Users\huynh.th\AppData\Local\Temp\ipykernel_21844\1616837008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["Date"] = pd.DatetimeIndex(options["Date"])
C:\Users\huynh.th\AppData\Local\Temp\ipykernel_21844\1616837008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["Maturity"] = pd.DatetimeIndex(options["Maturity"])


In [111]:
data.head()

,Date,Strike,Call,Maturity,Put
0,1412035200000000000,1850.0,1373.6,1418947200000000000,0.5
1,1412035200000000000,1900.0,1323.7,1418947200000000000,0.6
2,1412035200000000000,1950.0,1273.8,1418947200000000000,0.8
3,1412035200000000000,2000.0,1223.9,1418947200000000000,0.9
4,1412035200000000000,2050.0,1174.1,1418947200000000000,1.1


In [112]:
options

,Date,Strike,Call,Maturity,Put,T,r
38,2014-09-30,3175.0,126.8,2014-12-19,78.8,0.219178,0.02
39,2014-09-30,3200.0,110.9,2014-12-19,87.9,0.219178,0.02
40,2014-09-30,3225.0,96.1,2014-12-19,98.1,0.219178,0.02
41,2014-09-30,3250.0,82.3,2014-12-19,109.3,0.219178,0.02
42,2014-09-30,3275.0,69.6,2014-12-19,121.6,0.219178,0.02
342,2014-09-30,3175.0,171.0,2015-03-20,129.2,0.468493,0.02
343,2014-09-30,3200.0,156.1,2015-03-20,139.4,0.468493,0.02
344,2014-09-30,3225.0,142.0,2015-03-20,150.3,0.468493,0.02
345,2014-09-30,3250.0,128.5,2015-03-20,161.8,0.468493,0.02
346,2014-09-30,3275.0,115.8,2015-03-20,174.0,0.468493,0.02


In [101]:
T = (options["Maturity"] - options["Date"]).dt.days / 365.0
options["T"] = T 
options["r"] = 0.02

C:\Users\huynh.th\AppData\Local\Temp\ipykernel_21844\3597326862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["T"] = T
C:\Users\huynh.th\AppData\Local\Temp\ipykernel_21844\3597326862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["r"] = 0.02


In [128]:
i = 0
min_MSE = 500

In [127]:
def H93_error_function(p0):
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0 
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v ** 2:
        return 500.0
    se = []
    for row, option in options.iterrows():
        model_value = H93_call_value(
            S0,
            option["Strike"],
            option["T"],
            option["r"],
            kappa_v,
            theta_v,
            sigma_v,
            rho,
            v0
        )

        se.append((model_value - option["Call"]) ** 2)

    MSE = sum(se)/len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print("%4d |" % i, np.array(p0).round(2), "| %7.3f | %7.3f" % (MSE, min_MSE))
    i += 1
    return MSE

In [114]:
def H93_calibration_full():
    p0 = brute(
        H93_error_function,
        (
            (2.5, 10.6, 5.0),
            (0.01, 0.041, 0.01),
            (0.05, 0.251, 0.1),
            (-0.75, 0.01, 0.25),
            (0.01, 0.031, 0.01)
        ),
        finish = None
    )
    opt = fmin(
        H93_error_function, p0, xtol = .000001, ftol = .000001, maxiter =  750, maxfun = 900
    )
    return opt

In [129]:
H93_calibration_full()

   0 | [ 2.5   0.01  0.05 -0.75  0.01] | 820.892 | 500.000
  25 | [ 2.5   0.02  0.05 -0.75  0.02] |  23.864 |  21.568
  50 | [ 2.5   0.02  0.25 -0.75  0.03] |  89.655 |  21.568
  75 | [ 2.5   0.03  0.15 -0.5   0.01] | 193.283 |  21.568
 100 | [ 2.5   0.04  0.05 -0.5   0.02] | 176.340 |  21.568
 125 | [ 2.5   0.04  0.25 -0.5   0.03] | 486.965 |  21.568
 150 | [ 7.5   0.01  0.15 -0.25  0.01] | 840.337 |  21.568
 175 | [ 7.5   0.02  0.05 -0.25  0.02] |  24.810 |  21.568
 200 | [ 7.5   0.02  0.25 -0.25  0.03] |  24.834 |  21.568
 225 | [7.5  0.03 0.15 0.   0.01] | 110.936 |  21.568
 250 | [7.5  0.04 0.05 0.   0.02] | 540.183 |  21.568
 275 | [7.5  0.04 0.25 0.   0.03] | 783.222 |  21.568
 300 | [ 2.61  0.01  0.16 -0.76  0.03] |   8.046 |   7.795
 325 | [ 1.92  0.01  0.16 -0.92  0.02] |   6.301 |   6.146
 350 | [ 2.05  0.01  0.16 -0.89  0.03] |   6.151 |   6.145
 375 | [ 2.04  0.01  0.17 -0.87  0.03] |   6.097 |   6.075
 400 | [ 1.97  0.01  0.2  -0.83  0.03] |   6.002 |   5.996
 425 | [ 2.0

array([ 5.04735825,  0.01872573,  0.43477689, -0.44795621,  0.02728912])